In [2]:
import zipfile
import os

zip_path = '/content/satalite data-20250716T213334Z-1-001.zip'
extract_to = '/content/satalite_data-2'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print(f" Data extracted to {extract_to}")


 Data extracted to /content/satalite_data-2


In [6]:
train_img_dir = "/content/satalite_data-2/satalite data/data/images"
train_mask_dir = "/content/satalite_data-2/satalite data/data/labels"

In [7]:
import matplotlib.pyplot as plt


def display_image_and_mask(img_path, mask_path):
    img = Image.open(img_path)
    mask = Image.open(mask_path)

    # Plot the image and the mask side by side
    fig, axes = plt.subplots(1, 2, figsize=(10, 5))
    axes[0].imshow(img)
    axes[0].set_title('Image')
    axes[0].axis('off')

    axes[1].imshow(mask, cmap='gray')
    axes[1].set_title('Mask')
    axes[1].axis('off')

    plt.show()


In [8]:

# Get image and mask filenames and sort them to ensure matching
train_img_files = sorted(os.listdir(train_img_dir))
train_mask_files = sorted(os.listdir(train_mask_dir))

# Check that the number of images matches the number of masks
if len(train_img_files) != len(train_mask_files):
    print("Error: The number of images and masks do not match!")
else:
    # Display only the first 5 image-mask pairs
    for img_file, mask_file in zip(train_img_files[:5], train_mask_files[:5]):
        # Construct the full file paths
        img_path = os.path.join(train_img_dir, img_file)
        mask_path = os.path.join(train_mask_dir, mask_file)

        # Display the image and mask together
        display_image_and_mask(img_path, mask_path)

Error: The number of images and masks do not match!


In [9]:
!pip install tensorflow rasterio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 43.8 MB/s eta 0:00:00


In [10]:
import os
import numpy as np
import rasterio
import shutil
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Paths
images_folder = '/content/satalite_data-2/satalite data/data/images'
labels_folder = '/content/satalite_data-2/satalite data/data/labels'
augmented_folder = '/content/satalite_data-2/satalite data/data/augtrain'

# Create augmented folder
os.makedirs(augmented_folder, exist_ok=True)

# Count images & labels
image_files = sorted([f for f in os.listdir(images_folder) if f.endswith('.tif')])
num_labels = len([f for f in os.listdir(labels_folder) if os.path.isfile(os.path.join(labels_folder, f))])
num_images = len(image_files)

print(f"Found {num_images} images and {num_labels} labels.")

needed = num_labels - num_images
if needed <= 0:
    print("Images already >= labels. No need to augment.")
else:
    print(f"Need to generate {needed} augmented images.")

    # Augmentation settings
    datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=10,
        zoom_range=0.1,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    i = 0
    for img_file in image_files:
        img_path = os.path.join(images_folder, img_file)
        try:
            with rasterio.open(img_path) as src:
                img = src.read([1, 2, 3])  # read first 3 bands
                img = np.transpose(img, (1, 2, 0))  # channels last
        except Exception as e:
            print(f"Cannot read image: {img_file}, skipping. Reason: {e}")
            continue

        x = np.array(img, dtype='float32')
        x = x.reshape((1,) + x.shape)

        flow = datagen.flow(
            x, batch_size=1,
            save_to_dir=augmented_folder,
            save_prefix='aug',
            save_format='png'
        )

        for batch in flow:
            i += 1
            if i >= needed:
                break
        if i >= needed:
            break

    print(f"Generated exactly {i} augmented images.")

    # Copy original images to augmented folder
    for img_file in image_files:
        src_path = os.path.join(images_folder, img_file)
        dst_path = os.path.join(augmented_folder, img_file)
        shutil.copy2(src_path, dst_path)

    print(f"Copied {num_images} original images into augmented folder.")

    # Check final count
    final_files = [f for f in os.listdir(augmented_folder) if os.path.isfile(os.path.join(augmented_folder, f))]
    final_count = len(final_files)
    print(f"Final count in augmented folder: {final_count} files")
    print(f"Number of labels: {num_labels}")

    if final_count == num_labels:
        print("Success: Number of images now exactly matches number of labels!")
    else:
        print(f"Mismatch: total images ({final_count}) vs labels ({num_labels}). Check manually.")


Found 306 images and 456 labels.
Need to generate 150 augmented images.


/usr/local/lib/python3.11/dist-packages/rasterio/__init__.py:356: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


Generated exactly 150 augmented images.
Copied 306 original images into augmented folder.
Final count in augmented folder: 455 files
Number of labels: 456
Mismatch: total images (455) vs labels (456). Check manually.


In [12]:
import os

# Paths
images_folder = '/content/satalite_data-2/satalite data/data/images'
labels_folder = '/content/satalite_data-2/satalite data/data/labels'

# Count images (only tif files)
image_files = [f for f in os.listdir(images_folder) if f.endswith('.tif') and os.path.isfile(os.path.join(images_folder, f))]
label_files = [f for f in os.listdir(labels_folder) if os.path.isfile(os.path.join(labels_folder, f))]

num_images = len(image_files)
num_labels = len(label_files)

print(f" Number of images in folder: {num_images}")
print(f" Number of labels in folder: {num_labels}")


 Number of images in folder: 306
 Number of labels in folder: 456


In [13]:
import os
import numpy as np
import rasterio
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Paths
augmented_folder = '/content/satalite_data-2/satalite data/data/augtrain'
labels_folder = '/content/satalite_data-2/satalite data/data/labels'

# Count files
augmented_files = sorted([f for f in os.listdir(augmented_folder) if f.lower().endswith(('.tif', '.png', '.jpg'))])
num_augmented = len(augmented_files)

num_labels = len([f for f in os.listdir(labels_folder) if os.path.isfile(os.path.join(labels_folder, f))])

print(f"Found {num_augmented} images in augmented folder and {num_labels} labels.")

needed = num_labels - num_augmented
if needed <= 0:
    print("Images in augmented folder already >= labels. No need to augment.")
else:
    print(f"Need to generate {needed} more augmented images to match labels.")

    # Augmentation settings
    datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=10,
        zoom_range=0.1,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    i = 0
    # Use existing images in augmented folder for augmentation
    for img_file in augmented_files:
        img_path = os.path.join(augmented_folder, img_file)
        try:
            # Read tif with rasterio or fallback
            if img_file.lower().endswith('.tif'):
                with rasterio.open(img_path) as src:
                    img = src.read([1, 2, 3])  # first 3 bands
                    img = np.transpose(img, (1, 2, 0))
            else:
                from tensorflow.keras.preprocessing.image import load_img, img_to_array
                img = load_img(img_path)
                img = img_to_array(img)
        except Exception as e:
            print(f"Cannot read image: {img_file}, skipping. Reason: {e}")
            continue

        x = np.array(img, dtype='float32')
        x = x.reshape((1,) + x.shape)

        flow = datagen.flow(
            x, batch_size=1,
            save_to_dir=augmented_folder,
            save_prefix='aug2',
            save_format='png'
        )

        for batch in flow:
            i += 1
            if i >= needed:
                break
        if i >= needed:
            break

    print(f"Generated {i} new augmented images.")

    # Final check
    final_files = [f for f in os.listdir(augmented_folder) if os.path.isfile(os.path.join(augmented_folder, f))]
    final_count = len(final_files)
    print(f"Final count in augmented folder: {final_count} images")
    print(f"Number of labels: {num_labels}")

    if final_count == num_labels:
        print("Success: Number of images now exactly matches number of labels!")
    else:
        print(f"Mismatch: total images ({final_count}) vs labels ({num_labels}). Check manually.")


Found 455 images in augmented folder and 456 labels.
Need to generate 1 more augmented images to match labels.
Generated 1 new augmented images.
Final count in augmented folder: 456 images
Number of labels: 456
Success: Number of images now exactly matches number of labels!


In [18]:
train_img_dir2 = "/content/satalite_data-2/satalite data/data/augtrain"


**validation generator**

In [23]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

images = [os.path.join(train_img_dir2, file) for file in os.listdir(train_img_dir2)]
masks = [os.path.join(train_mask_dir, file) for file in os.listdir(train_mask_dir)]

data = pd.DataFrame({"image": images, "mask": masks})

train_df, val_df = train_test_split(data, train_size=0.8, shuffle=True, random_state=123)

In [24]:
image_datagen = ImageDataGenerator(rescale=1./255)
mask_datagen = ImageDataGenerator(rescale=1./255)

def create_generator(image_generator, mask_generator, batch_size=16):
    while True:
        for (img_batch, mask_batch) in zip(image_generator, mask_generator):
            yield img_batch, mask_batch

In [27]:
train_image_generator = image_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col="image",
    y_col=None,
    target_size=(128, 128),
    batch_size=16,
    class_mode=None,
    color_mode='rgb',
    shuffle=True,
    seed=42
)

train_mask_generator = mask_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col="mask",
    y_col=None,
    target_size=(128, 128),
    batch_size=16,
    class_mode=None,
    color_mode='grayscale',
    shuffle=True,
    seed=42
)

train_generator = create_generator(train_image_generator, train_mask_generator)


Found 364 validated image filenames.
Found 364 validated image filenames.


In [28]:
val_image_generator = image_datagen.flow_from_dataframe(
    dataframe=val_df,
    x_col="image",
    y_col=None,
    target_size=(128, 128),
    batch_size=16,
    class_mode=None,
    color_mode='rgb',
    shuffle=False,  # Validation data should not be shuffled
    seed=42
)

val_mask_generator = mask_datagen.flow_from_dataframe(
    dataframe=val_df,
    x_col="mask",
    y_col=None,
    target_size=(128, 128),
    batch_size=16,
    class_mode=None,
    color_mode='grayscale',
    shuffle=False,  # Validation data should not be shuffled
    seed=42
)

val_generator = create_generator(val_image_generator, val_mask_generator)

Found 92 validated image filenames.
Found 92 validated image filenames.


In [37]:
import rasterio
from PIL import Image
import os
import shutil

input_folder = '/content/satalite_data-2/satalite data/data/augtrain'
output_folder = '/content/satalite_data-2/satalite data/data/augt'
os.makedirs(output_folder, exist_ok=True)

for f in os.listdir(input_folder):
    if f.endswith('.tif'):
        img_path = os.path.join(input_folder, f)
        try:
            with rasterio.open(img_path) as src:
                print(f"\nProcessing {f}: has {src.count} bands.")
                if src.count >= 3:
                    img = src.read([1, 2, 3])
                    img = np.transpose(img, (1, 2, 0))
                elif src.count == 1:
                    img = src.read(1)  # read single band
                    img = np.stack([img]*3, axis=-1)  # duplicate into 3 channels
                else:
                    print(f"⚠ Skipping {f}: unexpected number of bands ({src.count})")
                    continue

                img = Image.fromarray(img.astype('uint8'))
                out_path = os.path.join(output_folder, f.replace('.tif', '.png'))
                img.save(out_path)
        except Exception as e:
            print(f"⚠ Cannot convert {f}: {e}")
    elif f.endswith(('.png', '.jpg', '.jpeg')):
        shutil.copy2(os.path.join(input_folder, f), os.path.join(output_folder, f))

print("\n✅ Finished converting tif images to png.")



Processing 291.tif: has 12 bands.

Processing 211.tif: has 12 bands.

Processing 70.tif: has 12 bands.

Processing 147.tif: has 12 bands.

Processing 219.tif: has 12 bands.

Processing 161.tif: has 12 bands.

Processing 259.tif: has 12 bands.

Processing 73.tif: has 12 bands.

Processing 75.tif: has 12 bands.

Processing 31.tif: has 12 bands.

Processing 191.tif: has 12 bands.

Processing 231.tif: has 12 bands.

Processing 91.tif: has 12 bands.

Processing 29.tif: has 12 bands.

Processing 260.tif: has 12 bands.

Processing 294.tif: has 12 bands.

Processing 197.tif: has 12 bands.

Processing 20.tif: has 12 bands.

Processing 101.tif: has 12 bands.

Processing 193.tif: has 12 bands.

Processing 230.tif: has 12 bands.

Processing 213.tif: has 12 bands.

Processing 222.tif: has 12 bands.

Processing 49.tif: has 12 bands.

Processing 204.tif: has 12 bands.

Processing 139.tif: has 12 bands.

Processing 76.tif: has 12 bands.

Processing 109.tif: has 12 bands.

Processing 102.tif: has 12 b

In [40]:

train_dir = '/content/satalite_data-2/satalite data/data/augt'
label_dir = '/content/satalite_data-2/satalite data/data/labels'

print("Images in train_dir:", len(os.listdir(train_dir)))
print("Images in label_dir:", len(os.listdir(label_dir)))
print("Sample image file names:", os.listdir(train_dir)[:5])
print("Sample label file names:", os.listdir(label_dir)[:5])

Images in train_dir: 456
Images in label_dir: 456
Sample image file names: ['135.png', 'aug_0_8436.png', '168.png', 'aug_0_4724.png', '303.png']
Sample label file names: ['135.png', '129_177.png', '168.png', '109_223.png', '53_43.png']


In [41]:
import os
from tensorflow.keras.utils import Sequence
import cv2
import numpy as np

class DataGenerator(Sequence):
    def __init__(self, image_files, mask_files, batch_size=16, img_size=(128,128)):
        self.image_files = image_files
        self.mask_files = mask_files
        self.batch_size = batch_size
        self.img_size = img_size

    def __len__(self):
        return int(np.ceil(len(self.image_files) / self.batch_size))

    def __getitem__(self, idx):
        batch_x = self.image_files[idx * self.batch_size : (idx + 1) * self.batch_size]
        batch_y = self.mask_files[idx * self.batch_size : (idx + 1) * self.batch_size]

        images, masks = [], []

        for img_path, mask_path in zip(batch_x, batch_y):
            img = cv2.imread(img_path)
            img = cv2.resize(img, self.img_size)
            img = img / 255.0

            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
            mask = cv2.resize(mask, self.img_size)
            mask = mask / 255.0
            mask = np.expand_dims(mask, axis=-1)

            images.append(img)
            masks.append(mask)

        return np.array(images), np.array(masks)


In [42]:
train_dir = '/content/satalite_data-2/satalite data/data/augt'
label_dir = '/content/satalite_data-2/satalite data/data/labels'

image_names = set([f for f in os.listdir(train_dir) if f.endswith('.png')])
label_names = set([f for f in os.listdir(label_dir) if f.endswith('.png')])

common_names = sorted(list(image_names & label_names))

print(f"Found {len(common_names)} matching images and masks.")

image_files = [os.path.join(train_dir, f) for f in common_names]
mask_files = [os.path.join(label_dir, f) for f in common_names]


Found 306 matching images and masks.


In [46]:
from tensorflow.keras import layers, Model, Input
from tensorflow.keras.optimizers import Adam

def unet_model(input_size=(128, 128, 3)):
    inputs = Input(input_size)

    # Encoder
    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    p1 = layers.MaxPooling2D((2, 2))(c1)

    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    p2 = layers.MaxPooling2D((2, 2))(c2)

    c3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    c3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c3)
    p3 = layers.MaxPooling2D((2, 2))(c3)

    c4 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(p3)
    c4 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(c4)
    p4 = layers.MaxPooling2D((2, 2))(c4)

    c5 = layers.Conv2D(1024, (3, 3), activation='relu', padding='same')(p4)
    c5 = layers.Conv2D(1024, (3, 3), activation='relu', padding='same')(c5)


    # Decoder
    u6 = layers.Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = layers.concatenate([u6, c4])
    c6 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(u6)
    c6 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(c6)

    u7 = layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = layers.concatenate([u7, c3])
    c7 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(u7)
    c7 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c7)

    u8 = layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = layers.concatenate([u8, c2])
    c8 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(u8)
    c8 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c8)

    u9 = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = layers.concatenate([u9, c1])
    c9 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(u9)
    c9 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c9)

    # Output layer
    outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(c9)

    model = Model(inputs=[inputs], outputs=[outputs])

    return model

In [47]:
batch_size = 16
img_size = (128,128)

train_generator = DataGenerator(image_files, mask_files, batch_size, img_size)

history = model.fit(
    train_generator,
    epochs=5
)

Epoch 1/5
20/20 ━━━━━━━━━━━━━━━━━━━━ 683s 34s/step - accuracy: 0.7318 - loss: 0.0169
Epoch 2/5
20/20 ━━━━━━━━━━━━━━━━━━━━ 688s 34s/step - accuracy: 0.7248 - loss: 0.0095
Epoch 3/5
20/20 ━━━━━━━━━━━━━━━━━━━━ 643s 32s/step - accuracy: 0.7216 - loss: 0.0095
Epoch 4/5
20/20 ━━━━━━━━━━━━━━━━━━━━ 633s 32s/step - accuracy: 0.7454 - loss: 0.0083
Epoch 5/5
20/20 ━━━━━━━━━━━━━━━━━━━━ 646s 32s/step - accuracy: 0.7287 - loss: 0.0086


**extra layers**

In [49]:
from tensorflow.keras import layers, Model, Input

def unet_15_16(input_size=(128, 128, 3)):
    inputs = Input(input_size)

    # Encoder: 15 conv layers (5 blocks × 3 conv each)
    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    p1 = layers.MaxPooling2D((2, 2))(c1)

    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    p2 = layers.MaxPooling2D((2, 2))(c2)

    c3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    c3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c3)
    c3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c3)
    p3 = layers.MaxPooling2D((2, 2))(c3)

    c4 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(p3)
    c4 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(c4)
    c4 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(c4)
    p4 = layers.MaxPooling2D((2, 2))(c4)

    c5 = layers.Conv2D(1024, (3, 3), activation='relu', padding='same')(p4)
    c5 = layers.Conv2D(1024, (3, 3), activation='relu', padding='same')(c5)
    c5 = layers.Conv2D(1024, (3, 3), activation='relu', padding='same')(c5)

    # Decoder: 16 conv layers (4 blocks × 4 conv each)
    u6 = layers.Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = layers.concatenate([u6, c4])
    c6 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(u6)
    c6 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(c6)
    c6 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(c6)
    c6 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(c6)

    u7 = layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = layers.concatenate([u7, c3])
    c7 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(u7)
    c7 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c7)
    c7 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c7)
    c7 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c7)

    u8 = layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = layers.concatenate([u8, c2])
    c8 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(u8)
    c8 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c8)
    c8 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c8)
    c8 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c8)

    u9 = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = layers.concatenate([u9, c1])
    c9 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(u9)
    c9 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c9)
    c9 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c9)
    c9 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c9)

    # Output layer
    outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(c9)

    model = Model(inputs=[inputs], outputs=[outputs])
    return model


In [50]:
model2 = unet_15_16(input_size=(128, 128, 3))
model2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model2.summary()




Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4       │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_76 (Conv2D)  │ (None, 128, 128,  │      1,792 │ input_layer_4[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_77 (Conv2D)  │ (None, 128, 128,  │     36,928 │ conv2d_76[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_78 (Conv2D)  │ (None, 128, 128,  │     36,928 │ conv2d_77[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_16    │ (None, 64, 64,    │          0 │ conv2d_78[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_79 (Conv2D)  │ (None, 64, 64,    │     73,856 │ max_pooling2d_16… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_80 (Conv2D)  │ (None, 64, 64,    │    147,584 │ conv2d_79[0][0]   │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_81 (Conv2D)  │ (None, 64, 64,    │    147,584 │ conv2d_80[0][0]   │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_17    │ (None, 32, 32,    │          0 │ conv2d_81[0][0]   │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_82 (Conv2D)  │ (None, 32, 32,    │    295,168 │ max_pooling2d_17… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_83 (Conv2D)  │ (None, 32, 32,    │    590,080 │ conv2d_82[0][0]   │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_84 (Conv2D)  │ (None, 32, 32,    │    590,080 │ conv2d_83[0][0]   │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_18    │ (None, 16, 16,    │          0 │ conv2d_84[0][0]   │
│ (MaxPooling2D)      │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_85 (Conv2D)  │ (None, 16, 16,    │  1,180,160 │ max_pooling2d_18… │
│                     │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_86 (Conv2D)  │ (None, 16, 16,    │  2,359,808 │ conv2d_85[0][0]   │
│                     │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_87 (Conv2D)  │ (None, 16, 16,    │  2,359,808 │ conv2d_86[0][0]   │
│                     │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_19    │ (None, 8, 8, 512) │          0 │ conv2d_87[0][0] 

 Total params: 49,873,153 (190.25 MB)

 Trainable params: 49,873,153 (190.25 MB)

 Non-trainable params: 0 (0.00 B)

In [53]:
batch_size = 12
img_size = (128,128)

train_generator = DataGenerator(image_files, mask_files, batch_size, img_size)

history = model2.fit(
    train_generator,
    epochs=10
)

Epoch 1/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 1084s 42s/step - accuracy: 0.7366 - loss: 0.0396
Epoch 2/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 1084s 41s/step - accuracy: 0.7274 - loss: 0.0101
Epoch 3/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 1053s 40s/step - accuracy: 0.7268 - loss: 0.0090
Epoch 4/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 1059s 41s/step - accuracy: 0.7471 - loss: 0.0081
Epoch 5/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 1070s 41s/step - accuracy: 0.7407 - loss: 0.0082
Epoch 6/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 1074s 41s/step - accuracy: 0.7183 - loss: 0.0087
Epoch 7/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 1107s 41s/step - accuracy: 0.7395 - loss: 0.0082
Epoch 8/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 1078s 41s/step - accuracy: 0.7396 - loss: 0.0081
Epoch 9/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 1050s 39s/step - accuracy: 0.7449 - loss: 0.0079
Epoch 10/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 1062s 41s/step - accuracy: 0.7257 - loss: 0.0085
